In [7]:
pip install -U numpy

Requirement already up-to-date: numpy in /opt/conda/lib/python3.7/site-packages (1.19.1)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier
from sklearn.model_selection import validation_curve

### Кривые валидации и обучения

In [9]:
#PATH_DATA = './data/data_part.json'
PATH_DATA = './data/data.json'
TRAIN_PERCENT = 10

In [10]:
dataRead = []
with open(PATH_DATA, 'r', encoding='utf-8') as jsonfile:    
    dataRead = json.load(jsonfile)
    
data = pd.DataFrame(data=dataRead)
data.head()

,Stylist,Visagiste,Unknown,Skilled,Advertising,Clip,Мale,Fashionshow,Videoshooting,Male,Photographer,Model,TVShow,Female,Photoshooting,Casting,Text
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,мужчин актер съемок клип съемк клип ищ геро му...
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,вниман москв т взлет ориентировочн девушк лет ...
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,х ф данил козловск пада аист мужчин лет рост р...
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,нов шанс попаст кастинг супер телевикторин уга...
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,кастинг июл девушек размер одежд парн работ ию...


In [11]:
columns = 'TVShow'
data_y = data[columns]
data_x = data['Text']

delimiter = int(len(data_y) * TRAIN_PERCENT / 100)

train_x = data_x[:delimiter]
test_x = data_x[delimiter:]
train_y = data_y[:delimiter]
test_y = data_y[delimiter:]

In [12]:
model_vectorizer = CountVectorizer()
vectorizer = model_vectorizer.fit(train_x)
feature_names = model_vectorizer.get_feature_names()

train_x = pd.DataFrame(data=vectorizer.transform(train_x).toarray(), columns=feature_names)
train_y = train_y 

test_x = pd.DataFrame(data=vectorizer.transform(test_x).toarray(), columns=feature_names)
test_y = test_y

In [13]:
X = pd.DataFrame(data=vectorizer.transform(data_x).toarray(), columns=feature_names)
y = data_y
train_x.shape, train_y.shape

((533, 4332), (533,))

In [ ]:
%%time
alphas = np.logspace(-2, 0, 20)
sgd_logit = SGDClassifier(loss='log', n_jobs=1, random_state=17)
logit_pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(degree=2)), 
                       ('sgd_logit', sgd_logit)])

val_train, val_test = validation_curve(logit_pipe, test_x, test_y,
                                       'sgd_logit__alpha', alphas, cv=5,
                                       scoring='roc_auc')

In [ ]:
def plot_with_err(x, data, **kwargs):
    mu, std = data.mean(1), data.std(1)
    lines = plt.plot(x, mu, '-', **kwargs)
    plt.fill_between(x, mu - std, mu + std, edgecolor='none',
                     facecolor=lines[0].get_color(), alpha=0.2)

plot_with_err(alphas, val_train, label='training scores')
plot_with_err(alphas, val_test, label='validation scores')
plt.xlabel(r'$\alpha$'); plt.ylabel('ROC AUC')
plt.legend();

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(degree=2, alpha=0.01):
    train_sizes = np.linspace(0.05, 1, 20)
    logit_pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(degree=degree)), 
                           ('sgd_logit', SGDClassifier(n_jobs=-1, random_state=17, alpha=alpha))])
    N_train, val_train, val_test = learning_curve(logit_pipe,
                                                  X, y, train_sizes=train_sizes, cv=5,
                                                  scoring='roc_auc')
    plot_with_err(N_train, val_train, label='training scores')
    plot_with_err(N_train, val_test, label='validation scores')
    plt.xlabel('Training Set Size'); plt.ylabel('AUC')
    plt.legend()

In [ ]:
plot_learning_curve(degree=2, alpha=10)

In [ ]:
plot_learning_curve(degree=2, alpha=0.05)